In [1]:
MSASL_TRAIN_PATH = "./MS-ASL/MSASL_train.json"
MSASL_VAL_PATH = "./MS-ASL/MSASL_val.json"
MSASL_TEST_PATH = "./MS-ASL/MSASL_test.json"

In [2]:
import json
import pandas as pd

with open(MSASL_TRAIN_PATH) as f:
    train_data = json.load(f)

with open(MSASL_VAL_PATH) as f:
    val_data = json.load(f)

with open(MSASL_TEST_PATH) as f:
    test_data = json.load(f)

train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)

train_df.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,match [light-a-MATCH],match,0.0,0,0,0,83,match light-a-MATCH,830,360.0,30.00,2.767,https://www.youtube.com/watch?v=C37R_Ix8-qs,match,"[0.05754461884498596, 0.21637457609176636, 1.0...",640.0,NaN
1,FAIL,fail,0.0,0,-1,0,74,FAIL,542,360.0,25.00,2.960,https://www.youtube.com/watch?v=PIsUJl8BN_I,fail,"[0.0657794177532196, 0.16717177629470825, 0.93...",480.0,NaN
2,laugh,laugh,0.0,4,26,0,31,SignSchool Laugh with Legs 2,312,360.0,29.97,1.034,www.youtube.com/watch?v=9FdHlMOnVjg,laugh,"[0.13188594579696655, 0.32334136962890625, 1.0...",640.0,NaN
3,BOOK,book,0.0,0,-1,0,66,BOOK(3),38,360.0,25.00,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.05569887161254883, 0.25173279643058777, 0.9...",480.0,NaN
4,sign-language,sign language,0.0,0,-1,0,75,SIGN-LANGUAGE-S-CLAW-F,848,360.0,29.97,2.502,www.youtube.com/watch?v=N2mG9ZKjrGA,sign language,"[0.03904399275779724, 0.24198183417320251, 1.0...",640.0,NaN


In [3]:
import os
from pytube import YouTube
from moviepy.video.io.VideoFileClip import VideoFileClip
from tqdm import tqdm

OUTPUT_DIR = 'msasl_clips'

os.makedirs(OUTPUT_DIR, exist_ok=True)

def download_and_trim_video(sample, save_dir):
    url = sample['url']
    label = sample['label']
    signer = sample['signer_id']
    start = float(sample['start_time'])
    end = float(sample['end_time'])

    try:
        yt = YouTube(url)
        stream = yt.streams.filter(file_extension='mp4', progressive=True).first()
        if stream is None:
            print(f"No suitable stream for: {url}")
            return None

        video_path = os.path.join(save_dir, f"{label}_{signer}.mp4")
        temp_path = os.path.join(save_dir, f"temp_{label}_{signer}.mp4")
        
        stream.download(filename=temp_path)

        clip = VideoFileClip(temp_path).subclip(start, end)
        clip.write_videofile(video_path, audio=False, verbose=False, logger=None)
        clip.close()
        os.remove(temp_path)

        return video_path

    except Exception as e:
        print(f"Failed to download or process {url}: {e}")
        return None

In [ ]:
downloaded = []

for sample in tqdm(train_data[:100]): 
    path = download_and_trim_video(sample, OUTPUT_DIR)
    if path:
        downloaded.append(path)
    if len(downloaded) >= 10:
        break

  1%|█                                                                                                          | 1/100 [00:01<02:27,  1.49s/it]

Failed to download or process https://www.youtube.com/watch?v=C37R_Ix8-qs: HTTP Error 400: Bad Request


  2%|██▏                                                                                                        | 2/100 [00:03<02:29,  1.52s/it]

Failed to download or process https://www.youtube.com/watch?v=PIsUJl8BN_I: HTTP Error 400: Bad Request


  3%|███▏                                                                                                       | 3/100 [00:03<01:52,  1.16s/it]

...

Failed to download or process https://www.youtube.com/watch?v=htsdwxJ-fTo: HTTP Error 400: Bad Request


 85%|██████████████████████████████████████████████████████████████████████████████████████████                | 85/100 [01:58<00:22,  1.49s/it]

Failed to download or process https://www.youtube.com/watch?v=htsdwxJ-fTo: HTTP Error 400: Bad Request


 86%|███████████████████████████████████████████████████████████████████████████████████████████▏              | 86/100 [01:59<00:20,  1.45s/it]

Failed to download or process https://www.youtube.com/watch?v=htsdwxJ-fTo: HTTP Error 400: Bad Request
